<a href="https://colab.research.google.com/github/LukasRodriguess/process_automation/blob/main/Automatiza%C3%A7%C3%A3o_de_processos_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatização de processos

---

A ideia desse projeto é tornar operacional um processo antes executado manualmente,  utilizando para isso da tecnologia e da integração de sistemas e dados.

Estarei utilizando Python, Requets, SQLite



##### Libraries ↓

In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import yfinance as yf #API Yahoo finance

#libs for the operating system ↓
import os
import platform
#import win32com.client as win32

#libs for connection ↓
import requests
import socket

#libs to ignore warnings ↓
import warnings
warnings.filterwarnings('ignore')



#####Rotinas Iniciais

In [2]:
routine_id = 1
routine_name = 'Fluxo de dados - Corretora'

#Identify user information
def identify_user():
  user = os.environ.get('USERNAME') #Catch user name
  machine = platform.node() #Catch machine name
  operating_system = platform.platform() #Catch operating System

  return (user, machine, operating_system)

#---------------------------------------------------------------------------------
#Identify the beginning of the routine
def start_routine():
  date_start = datetime.today().date()
  hour_start = datetime.now()

  return (date_start, hour_start)

#---------------------------------------------------------------------------------
#identify the end of the routine
def end_routine():
  date_end = datetime.today().date()
  hour_end = datetime.now()

  return (date_end, hour_end)

#---------------------------------------------------------------------------------
def check_connection():
  URL = "https://www.google.com"

  time_out = 5
  try:
    requests.get(URL, timeout=time_out)
    return True

  except:
    return False

#---------------------------------------------------------------------------------
def identify_ip():
  try:
    local_ip = socket.gethostbyname(socket.gethostname())
    return local_ip
  except:
    return False


#---------------------------------------------------------------------------------


list_user = identify_user()
list_start = start_routine()
list_end = end_routine()
connection_net = check_connection()
check_ip = identify_ip()

operational_error = ''

In [ ]:
try:
  yf.pdr_override()
  data_base = web.get_data_yahoo('ARZZ3.SA', start='2024-01-01') #get data
  eixo_plot = data_base['Close']

  #Configurando grafico ↓
  plt.figure( figsize=(15,6) ) #tamanho
  plt.style.use('ggplot') #estilo

  plt.plot(eixo_plot) #plot do grafico

  #labels e titulos ↓
  plt.title("Analise Arezzo") #title
  plt.xlabel('Periodo') #Eixo X
  plt.ylabel('Preço fechamento')#Eixo y

  plt.savefig('Indicador_Ações.png') #salvando grafico em png

except:
  operational_error = 'Execution_Error'

In [4]:
#list_end = end_routine()
dictionary = {
    'Id Routine': routine_id,
    'Name_Routine': routine_name,
    'User': list_user[0],
    'Machine': list_user[1],
    'Operation_System': list_user[2],
    'Date_Start': list_start[0],
    'Hour_Start': list_user[1],
    'Try_connection': connection_net,
    'Local_Ip': check_ip,
    'Date_end': list_end[0],
    'Hour_end': list_end[1],
    'Timex_Ex': (list_end[1] - list_start[1]),
    'Error': operational_error
}

In [5]:
table = pd.DataFrame(dictionary, index=[0])
table

,Id Routine,Name_Routine,User,Machine,Operation_System,Date_Start,Hour_Start,Try_connection,Local_Ip,Date_end,Hour_end,Timex_Ex,Error
0,1,Fluxo de dados - Corretora,None,592b370c17b0,Linux-6.1.85+-x86_64-with-glibc2.35,2024-05-29,592b370c17b0,True,172.28.0.12,2024-05-29,2024-05-29 13:52:49.711396,0 days 00:00:00.000076,


#####test

#####Operacional

#####Finalização